In [1]:
import pandas as pd
import numpy as np
import math

SOLVER_MILO = "highs"
SOLVER_MINLO = "ipopt"

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["coin", "highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

AMPL Development Version 20240404 (MSVC 19.38.33135.0, 64-bit)
Demo license with maintenance expiring 20260131.
Using license file "c:\Users\thuduong\Anaconda3\envs\optima\Lib\site-packages\ampl_module_base\bin\ampl.lic".



### A. Optimize on selected stock then compare the result (base case)

In [ ]:
# DATA TEST 1
### uu tien cat het 1 finished goods trong 1 cuon thep
### -> need cut chuyen thanh duong truoc khi sang finished goods khac
###
### uu tien trim-loss hay weight loss thap hon
### ->
###
### ---> chon coil cat duoc het nhieu cac don va co trim loss that

### B. Optimize with test data - Dont know the solution at first
#### Try test with boundary on % need cut 
- Case 0: + 30% need cut
- Case 1: need cut + 300% forecast
- Case 2: need cut + 600% forecast

#### Input Parameters

In [2]:
## PARAMETER - CASE 1
PARAMS = {"warehouse": "HSC"
          ,"spec_name": "JSH270C-PO" # yeu cau chuan hoa du lieu OP - PO
          ,"thickness": 1.6
          ,"maker" : "CSVC"
          ,"stock_ratio": { #get from app 
                    "default": 2
                }
        #   ,"forecast_scenario":
          }


margin_dict = { #save margin_dict in azure env
    "thickness_2.6": {
        "thickness": 2.6,
        "margin": 10
    },
    "thickness_2": {
        "thickness": 2,
        "margin": 8
    },
    "thickness_1.6": {
        "thickness": 1.6,
        "margin": 6
    }
}

In [3]:
# GET ALL PARAMS
# print("Thickness of 2.6:", margin_dict["Thickness_2.6"]["margin"])
MIN_MARGIN = margin_dict[f"thickness_{PARAMS["thickness"]}"]["margin"]
print(f"MIN_MARGIN:{MIN_MARGIN}")

BOUND_KEY = next(iter(PARAMS['stock_ratio']))
BOUND_VALUE = PARAMS['stock_ratio'][BOUND_KEY]
print(f"BOUND_VALUE:{BOUND_VALUE}")

MIN_MARGIN:6
BOUND_VALUE:2


#### Input & Process Data

In [4]:
from codes.process_df import filter_stock_excel_to_dict
file_path = "data/test_mc_df.xlsx"
stock_key, value_columns = "inventory_id", ['receiving_date','status',"width", "weight"]
stocks = filter_stock_excel_to_dict(file_path, stock_key, value_columns, PARAMS, PRIORITY = "CASE_1")
print(len(stocks))
# list of stock >> list of need cut PO
# Improve: need rules to pick up stock to the pool

2


In [5]:
from codes.process_df import filter_finish_excel_to_dict

file_path = "data/test_finish_df_Jul2022.xlsx"
finish_key, value_columns = "order_id", ["width", "need_cut", f"upper_bound_{BOUND_KEY}", "fc1", "fc2", "fc3"]
finish = filter_finish_excel_to_dict(file_path, finish_key, value_columns, PARAMS, BOUND_KEY, BOUND_VALUE)
len(finish)

# nen loc list need cut voi so am truoc,
#Improve: uu tien thu solion voi FG co width lon truoc

10

#### Naive patterns and combination

In [6]:
from codes.create_patterns import *
naive_patterns = make_patterns_by_weight_width(stocks, finish, BOUND_KEY, MIN_MARGIN)

# display(naive_patterns)
len(naive_patterns)
print(stocks.keys())
# uu tien so need cut va so can cua mother coil ko gap nhau qua 5 (X) lan. de combination ko loi

dict_keys(['S813', 'S885'])


In [7]:
def generate_cut_combinations_with_timeout1(stock_id, min_c_values, max_c_values, pattern, TIMEOUT):

    def generate_combinations_util(keys, current_combination, start_time, TIMEOUT):

        if not keys:
            if any(current_combination.values()):  # Check if any 'F' value is non-zero, remove the case all zeros
                pattern.append({'stock': stock_id, 'cuts': current_combination})
            return
        
        key = keys[0]
        min_c = min_c_values.get(key, 0) #retrieves the value associated, 0 if not found
        max_c = max_c_values.get(key, 0)

        for c in range(max_c, min_c - 1, -1):
            # Check if the timeout has been exceeded
            if time.time() - start_time >= TIMEOUT:
                print(f"key{key}, timeout{time.time() - start_time}")
                return pattern  # Return the pattern if timeout occurs
            else:
                new_combination = current_combination.copy()
                new_combination[key] = c
                generate_combinations_util(keys[1:], new_combination, start_time, TIMEOUT)

    # keys = set(min_c_values.keys()) | set(max_c_values.keys()) # converts any iterable into a set without order
    reorderd_max_c = change_order_dict(max_c_values)
    keys = list(reorderd_max_c.keys())

    import time
    start_time = time.time()
    generate_combinations_util(list(keys), {},start_time, TIMEOUT)

In [8]:
s = "S885"
TIMEOUT = 30
patterns = []
min_cuts_dict, max_cuts_dict  = ap_stock_bound(naive_patterns,finish,s)
print(f"Start create combination for stock {s}")
generate_cut_combinations_with_timeout1(s, min_cuts_dict, max_cuts_dict, patterns, TIMEOUT)

Start create combination for stock S885
keyF14, timeout31.399585485458374
keyF15, timeout31.407603979110718
keyF17, timeout31.407603979110718
keyF18, timeout31.407603979110718
keyF10, timeout31.408599853515625
keyF11, timeout31.408599853515625
keyF12, timeout31.408599853515625
keyF13, timeout31.408599853515625


In [ ]:
from codes.optima_sol import cut_patterns_by_stock
opt_patterns = cut_patterns_by_stock(
                                     stocks[s]["width"]
                                     ,stocks[s]["weight"] 
                                     ,finish 
                                     ,patterns
                                     ,BOUND_KEY
                                     )

In [ ]:
# from codes.errors_handling import run_with_timeout
from codes.optima_sol import cut_patterns_by_stock

for s in stocks.keys():
    # create pattern by stocks
    patterns = []
    min_cuts_dict, max_cuts_dict  = ap_stock_bound(naive_patterns,finish,s)
    print(f"Start create combination for stock {s}")
    generate_cut_combinations_with_timeout(s, min_cuts_dict, max_cuts_dict, patterns, TIMEOUT = 60)
    if patterns is None:
        print("Null value")
    else:
        opt_patterns = cut_patterns_by_stock(
                                     stocks[s]["width"]
                                     ,stocks[s]["weight"] 
                                     ,finish 
                                     ,patterns
                                     ,BOUND_KEY
                                     )

        if len(opt_patterns) == 0:
            print(f"No optimal solution with stock {s}")
            print("=================================")
        else:
            print(f"Solution for stock {s}")
            for p in opt_patterns:
                print(f"take pattern {p}")
                cuts_dict= patterns[p]['cuts']
                print(cuts_dict)
                trim_loss = stocks[s]["width"] - sum([finish[f]["width"]*cuts_dict[f] for f in finish.keys()])
                weight_loss = trim_loss * stocks[s]["weight"]/stocks[s]["width"]
                print(f"trim loss :{trim_loss}, weight loss: {weight_loss}")
                print("*****")

            print("=================================")


#### Optimization

In [ ]:
from codes.errors_handling import run_with_timeout
from codes.optima_sol import cut_patterns_by_stock
timeout_limit = 40

for s in stocks.keys():
    # create pattern by stocks
    patterns = []
    max_cuts_dict, min_cuts_dict = ap_stock_bound(naive_patterns,finish,s)
    print(f"Start create combination for stock {s}")
    patterns = run_with_timeout(generate_cut_combinations, args=(s, min_cuts_dict, max_cuts_dict, patterns), timeout=timeout_limit)
    if patterns is None:
        print("Function timed out")
    else:
 
        opt_patterns = cut_patterns_by_stock(
                                     stocks[s]["width"]
                                     ,stocks[s]["weight"] 
                                     ,finish 
                                     ,patterns
                                     )

        if len(opt_patterns) == 0:
            print(f"No optimal solution with stock {s}")
            print("=================================")
        else:
            print(f"Solution for stock {s}")
            for p in opt_patterns:
                print(f"take pattern {p}")
                cuts_dict= patterns[p]['cuts']
                print(cuts_dict)
                trim_loss = stocks[s]["width"] - sum([finish[f]["width"]*cuts_dict[f] for f in finish.keys()])
                weight_loss = trim_loss * stocks[s]["weight"]/stocks[s]["width"]
                print(f"trim loss :{trim_loss}, weight loss: {weight_loss}")
                print("*****")

            print("=================================")
